In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import lsst.sims.maf.db as db
import lsst.sims.maf.utils as utils
import lsst.sims.maf.metrics as metrics
import lsst.sims.maf.slicers as slicers
import lsst.sims.maf.metricBundles as metricBundles
from mafContrib import Plasticc_metric, plasticc_slicer
from lsst.sims.utils import _healbin
import healpy as hp

/Users/yoachim/lsstp3/python/miniconda3-4.3.21/lib/python3.6/site-packages/matplotlib/__init__.py:1405: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

  warnings.warn(_use_error_msg)
/Users/yoachim/lsstp3/python/miniconda3-4.3.21/lib/python3.6/site-packages/matplotlib/__init__.py:1405: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

  warnings.warn(_use_error_msg)


In [2]:
runName = 'baseline_1exp_pairsmix_10yrs'
#This next command assumes you have downloaded baseline2018a.db into the tutorials directory in this repo.
#Change this path according to your datbase and where it is saved.
opsdb = db.OpsimDatabase('baseline_1exp_pairsmix_10yrs.db')
outDir = 'temp'
resultsDb = db.ResultsDb(outDir=outDir)

In [3]:
bundleList = []
sql=''
slicer1 = plasticc_slicer(model='SNIa-normal')
slicer2 = plasticc_slicer(model='KN', seed=43)


In [4]:
metric = Plasticc_metric(metricName='SNIa')
summary_stats = [metrics.SumMetric()]
bundle = metricBundles.MetricBundle(metric, slicer1, sql, runName=runName, summaryMetrics=summary_stats)
bundleList.append(bundle)

metric = Plasticc_metric(metricName='KN')
bundle = metricBundles.MetricBundle(metric, slicer2, sql, runName=runName, summaryMetrics=summary_stats)
bundleList.append(bundle)



In [ ]:
bundleDict = metricBundles.makeBundlesDictFromList(bundleList)
bgroup = metricBundles.MetricBundleGroup(bundleDict, opsdb, outDir=outDir, resultsDb=resultsDb)
bgroup.runAll()

Querying database SummaryAllProps with no constraint for columns ['fieldDec', 'fieldRA', 'observationStartMJD', 'filter', 'fiveSigmaDepth'].
Found 2609253 visits
Running:  ['baseline_1exp_pairsmix_10yrs_SNIa_USER']


/Users/yoachim/gitRepos/sims_maf_contrib/mafContrib/plasticcMetrics.py:101: RuntimeWarning: invalid value encountered in log10
  result[infilt] = -2.5*np.log10(np.interp(mjds[infilt], lc_mjd, plc[filttrans[filtername]]['fluxcal'], left=-1, right=-1)) + zp


Completed metric generation.
Running:  ['baseline_1exp_pairsmix_10yrs_KN_USER']
Completed metric generation.
Running reduce methods.
Running summary statistics.
Completed.


In [ ]:
bgroup.plotAll(closefigs=False)

Plotting figures with "" constraint now.


/Users/yoachim/gitRepos/sims_maf/python/lsst/sims/maf/plots/plotHandler.py:518: UserWarning: Cannot plot object metric values with this plotter.
  warnings.warn('Cannot plot object metric values with this plotter.')
/Users/yoachim/gitRepos/sims_maf/python/lsst/sims/maf/utils/mafUtils.py:58: RuntimeWarning: divide by zero encountered in double_scalars
  nbins = (binmax - binmin) / binwidth
/Users/yoachim/gitRepos/sims_maf/python/lsst/sims/maf/utils/mafUtils.py:60: UserWarning: Optimal bin calculation tried to make inf bins, returning 200
  warnings.warn('Optimal bin calculation tried to make %.0f bins, returning %i'%(nbins, nbinMax))


Plotting complete.


In [ ]:
for key in bundleDict:
    print(key, bundleDict[key].summaryValues)


In [ ]:
ack = _healbin(slicer1.slicePoints['ra'],slicer1.slicePoints['dec'],
               bundleDict['SNIa_WellFit'].metricValues, nside=8,
               reduceFunc=np.sum)
              

In [ ]:
hp.mollview(ack)

In [ ]:
plt.plot(slicer2.slicePoints['plc'][0]['r']['mjd'], -2.5*np.log10(slicer2.slicePoints['plc'][0]['r']['fluxcal'])+27.5 )
plt.ylim([35,19])